# Конструирование суррогатной модели

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Основная задача этой главы - изучить отображение $y = f(x)$ , производящееся в так называемом "чёрном ящике", который скрывает физику этого отображения, преобразующего вектор __x__ в скалярную величину $y$ . Этот черный ящик может принимать форму физического или компьютерного эксперимента, например, кода, реализующего метод конечных элементов, который вычисляет максимальное напряжение (f) для заданных размеров изделия (__x__). Общий метод решения данной проблемы состоит следующем: сначала собираются выходные значения $y^{(1)},y^{(2)},...,y^{(n)}$, каждое из которых является результатом действия отображения $f$ на входные данные $x^{(1)},x^{(2)},...,x^{(n)}$, а затем (на основе наблюдений) ищется функция $\hat{f}(x)$, которая описывает отображение чёрного ящика $f$ наилучшим образом.  
В этой главе мы обсудим основы и некоторые технические детали ряда конкретных типов суррогатных моделей, способных решить поставленную задачу. Однако начнём мы с обсуждения ключевых этапов процесса построения суррогатной модели.

## Процесс Моделирования

### Первая стадия: Подготовка данных и выбор Подхода к моделированию

В разделе 1 рассматриваются два предварительных шага этого этапа. Первый - это идентификация: с помощью небольшого числа наблюдений определить те параметры входных данных, которые оказывают существенное влияние на $f$; то есть происходит определение кратчайшего вектора проектной переменной $x = \{x_1, x_2,..., x_k\}^T$, который, в рассматриваемых диапозонах всех своих компонент может сильно влиять на поведение чёрного ящика. Если бы чёрный ящик был электронным устройством, оснащенным большим набором элементов управления, то этот шаг сводился бы к выявлению $k$ элементов управления, которые при изменении влияют на его поведение. Эта операция осложнена возможными взаимодействиями между элементами управления. На этом этапе также необходимо установить диапазоны различных проектных переменных.  

Второй шаг - сбор $n$ штук векторов длины $k$ в список $X = \{ x^{(1)}, x^{(2)},...,x^{(n)} \}^T$, таким образом, чтобы этот набор максимально полно представлял собой пространство проектирования. Основная проблема в том, что n часто невелико, т.к. оно ограничено вычислительной и временной стоимостью, приходящейся на каждое наблюдение $f(x^{(i)})$.  
Возможно, здесь стоит повторить, что на этом шаге неплохо масштабировать x в единичный куб $[0,1]^k$, что может упростить некоторые последующие математические вычисления и избавить нас от множества проблем многомерного масштабирования.  

Поскольку в главе 1 изложен ряд методов для выполнения вышеуказанного, здесь мы рассмотрим следующую фазу процесса - фактическую попытку изучения $f$ с помощью пар данных $\{(x^{(1)},y^{(1)}),(x^{(2)},y^{(2)}), ...(x^{(n)},y^{(n)})\}$. Этот так называемый _процесс обучения под наблюдением или на основе примеров_ (___обучениие с учителем___), по сути, представляет собой поиск в пространстве всех возможных функций $\hat{f}$, которые воспроизводили бы наблюдения $f$.  

Это пространство бесконечно. В конце концов, можно нарисовать любое количество (гипер)поверхностей, прохождящих в определенном диапазоне (с учетом экспериментальной ошибки) известных наблюдений. Однако подавляющее большинство из них будет очень плохо обобщаться; то есть они будут практически бесполезны для прогнозирования ответов на новых данных, в чем и заключается цель данной задачи.

Рассмотрим несколько экстремальный пример функции "иголка(иголки) в стоге сена":


$$ \hat{f} = \begin{cases}
   y^{(1)} &\text{if $x = x^{(1)}$}\\
   y^{(2)} &\text{if $x = x^{(2)}$}\\
   ...\\
   y^{(n)} &\text{if $x = x^{(n)}$}\\
   0 &\text{иначе}\\ 
 \end{cases} $$

Очевидно, что, хотя все обучающие данные могут быть воспроизведены с помощью этого предиктора, но есть, по крайней мере, в отношении большинства инженерных функций, что-то сильно противоречащее интуиции и тревожащее в том, что он предсказывает 0 везде, кроме обучающих точек. Конечно, есть небольшая вероятность того, что функция действительно выглядит как (1), и по какой-то необычайной случайности нам довелось исследовать её именно там, где находятся все иглы, но это маловероятно.

Можно было бы придумать множество других, возможно, менее надуманных примеров, которые также кажутся какими-то неестественными и, более того, плохо обобщаются. В конечном счете все это говорит о том, что нам нужны какие-то систематические инструменты фильтрации таких бессмысленных предсказателей. Некоторые ученые занимают здесь Байесовскую позицию, например, философия, отстаиваемая Расмуссеном и Уильямсом (2006), заключается в том, чтобы "заранее дать вероятность каждой возможной функции, причём более высокие вероятности дать функциям, которые мы считаем более вероятными, например, потому что они более гладкие, чем другие функции".  

Подход, который мы будем использовать в дальнейшем, заключается в том, чтобы заранее опредлить структуру $\hat{f}$ (например, полином третей степени), встроить её в алгоритм моделирования и выполнить поиск по пространству её параметров для настройки приближения к наблюдениям. Например, рассмотрим одну из простейших возможных моделей: $\hat{f}(x,w) = w^T * x + v$. Изучение f с помощью этой модели подразумевает, что мы определились с ее структурой – это будет гиперплоскость – и процесс подгонки модели состоит в нахождении $k + 1$ параметров (вектор наклона w и перехват v), для которых значение выражения $w^T * x + v$ наилучшим образом соответствует данным (это будет выполнено на второй стадии).  

Все вышесказанное часто дополнительно осложняется наличием шума в наблюдаемых откликах (будем считать, что проектные вектора \textbf{x} никоим образом не повреждены). Мы обсуждали природу этого шума в начале главы 1. Здесь мы концентрируемся на изучении таких данных, для которых иногда присутсвует вероятность ___перенасыщения___.
Перенасыщение происходит, когда модель в некотором смысле слишком гибкая и слишком точно соответствует обучающим данным, то есть соответствует шуму, а также фактическому базовому поведению, которое мы пытаемся смоделировать. На второй стадии процесса суррогатного строительства эта проблема _управления сложностью_ решается с помощью процесса оценки параметров модели фиксированной конструкции, но здесь, на этапе выбора типа модели, также требуется некоторое предвидение в этом направлении.  

Обычно этот шаг связан с физическими соображениями; то есть выбор метода моделирования зависит от наших ожиданий того, как может выглядеть лежащий в основе ответ. Например, если у нас есть некоторые наблюдения за напряжением в упруго деформированном твердом теле в ответ на небольшие деформации, имеет смысл моделировать напряжение с помощью простого линейного приближения.  
Если такое понимание физики недоступно и мы не сможем учесть, скажем, линейность данных на данном этапе, мы в конечном итоге примем сложную, чрезмерно гибкую модель. Это не будет концом света (в конце концов, этап оценки параметров, как мы надеемся, "линеаризует" аппроксимацию путем соответствующего выбора параметров, управляющих ее формой), но мы упустим возможность получить алгебраически простую, надежную модель.  

И наоборот, если мы ошибочно предположим, что данные получены, скажем, из базового квадратичного процесса, а на самом деле истинная функция $f$ имеет множество пиков и впадин, на этапе оценки параметров мы не сможем компенсировать неудачно выбранную модель. Квадратичная просто будет слишком жесткой, чтобы соответствовать мультимодальной функции, каковы бы ни были ее параметры.

### Вторая стадия: Оценка параметров и обучение

Давайте предположим, что на первой стадии мы определили $k$ проектных переменных, получили набор обучающих данных и выбрали общую структуру модели $\hat{f}(x, w)$, где точная форма модели определяется набором параметров $w$. Теперь мы сталкиваемся с проблемой оценки параметров: как нам выбрать такие $w$, чтобы модель наилучшим образом соответствовала данным? Существует много критериев оценки, но здесь мы обсудим два.

#### Метод Максимального Правдоподобия

Если даны набор параметров $w$ и сама модель $\hat{f} (x, w)$, то мы можем вычислить вероятность того, что набор данных $\{(x^{(1)},y^{(1)} \pm \epsilon),(x^{(2)},y^{(2)} \pm \epsilon),...,(x^{(n)},y^{(n)} \pm \epsilon)\}$ будет получен из $\hat{f}$ (где $\epsilon$ - некоторая небольшая постоянная погрешность вокруг каждой точки). Мы можем предположить, что ошибки независимо случайным образом распределены в соответствии с нормальным распределением со стандартным отклонением $\sigma$, тогда вероятность получения данного набора данных равна  

$$P = \frac{1}{(2\pi \sigma^2)^{n/2}} \prod\limits_{i=1}^n \Bigr\{ exp \Bigr[ - \frac{1}{2} \Bigr( \frac{y^{(i)} - \hat{f}(x,w)} {\sigma}\Bigl)^2 \Bigl] \epsilon \Bigl\}$$

В этом и заключается метод максимального правдоподобия: мы предполагаем, что результаты, полученные нашей выборкой, наиболее вероятны (являются мат.ожиданием). Тогда нам нужно максимизировать значение этого выражения. Для упрощения вычислений - прологорифмируем и найдём минумум выражения с обратным знаком, приэтом избавимся от незначимых констант:  

$$\min_w \sum\limits_{i=1}^n \frac{\Bigr[y^{(i)} - \hat{f}(x,w)\Bigl]^2}{2 \sigma^2} - n \ln \epsilon$$

Заметим, что если мы предполагаем $\sigma$ и $\epsilon$ постоянными, то выражение (3) упрощается и принимает хорошо знакомый вид _метода наименьших квадратов_ :  

$$\min_w \sum\limits_{i=1}^n \Bigr[y^{(i)} - \hat{f}(x,w)\Bigl]^2$$

#### Кросс-валидация (скользящий контроль или метод перекрёстной проверки)

Перекрёстная проверка состоит из нескольких этапов: 
1. разбиение данных (случайным образом) на $q$ примерно равных по объёму подмножеств; 
2. удаление одного из этих подмножеств; 
3. обучнеие модели на оставшихся $q − 1$ подмножествах. 
4. Вычисление функции потерь $L$, которая измеряет ошибку между полученным на этапе 3) предиктором и точками в подмножестве, которое мы удалили на этапе 2).

Затем возвращаемся к этапу 2) (вернув удалённое подмножество) и удаляем другое ещё не тронутое подмножество. Снова вычисляем функцию потерь $L$ и суммируем за все $q$ итераций.  

Более формально, если отображение $\zeta$ : $\{1,....n\} \rightarrow \{1,...,q\}$ описывает распределение $n$ обучающих точек в одно из $q$ подмножеств, а $\hat{f}^{-\zeta(i)}(x)$ - значение предиктора (в точке $x$), полученное путем удаления подмножества $\zeta(i)$ (т.е. подмножества, которому соответствует наблюдение $i$), то мера перекрёстной проверки, которую мы используем здесь в качестве оценки ошибки прогнозирования, равна  

$$\varepsilon_{cv}(w) = \frac{1}{n} \sum\limits_{i=1}^n L \Bigr[y^{(i)}, \hat{f}^{-\zeta(i)}(x^{(i)},w)  \Bigl]$$


Функцию потерь $L$ можно ввести, как квадрат ошибки, тогда можно переписать уравнение (5) так:  

$$\varepsilon_{cv}(w) = \frac{1}{n} \sum\limits_{i=1}^n \Bigr[y^{(i)} - \hat{f}^{-\zeta(i)}(x^{(i)},w)  \Bigl]^2$$

В какой степени уравнение (6) является объективной оценкой истинной ошибки прогнозирования, зависит от выбора $q$. Можно показать, что если $q = n$, то $\varepsilon_{cv}$ является почти точной оценкой истинной ошибки. Однако дисперсия этой _меры исключения_ может быть очень высокой из-за того, что $n$ подмножеств очень похожи друг на друга. Хасти и др. (2001) рекомендуют компромиссные значения $q = 5$ или $q = 10$. С практической точки зрения, использование меньшего количества подмножеств дает дополнительный бонус в виде снижения вычислительных затрат на процесс перекрестной проверки за счет сокращения количества моделей, которые необходимо подставлять.

### Третья стадия: Тестирование модели

Если данных наблюдений достаточно, то в начале следует отложить случайно выбранное подмножество (обычно рекомендуют брать около $\frac{n}{4}$ пар значений $x \rightarrow y$) для целей тестирования модели. Эти наблюдения не должны использоваться ни на первой, ни на второй стадиях, поскольку их единственная цель состоит в том, чтобы позволить нам оценить ошибку тестирования (на основе разницы между истинными и приближенными значениями функций на испытательных площадках) после построения модели.  

Отметим, что в идеале оценка параметров модели (на второй стадии) также должна основываться на отдельном подмножестве, но данные наблюдений редко бывают настолько обильными, чтобы позволить эту роскошь (если функция очень дешева для оценки, то нам может и не понадобиться суррогатная модель вообще). Если данные доступны для тестирования модели и нашей основной целью является глобально точная модель, то мы используем либо метрику среднеквадратичной ошибки (RMSE), либо коэффициента корреляции $r^2$. А чтобы протестировать модель, мы просто берём набор тестовых данных размером $n_t$ и набор прогнозов в местах расположения тестовых данных и вычисляем либо  

$$\text{RMSE} = \sqrt{\frac{\sum\limits_{i=0}^{n_t}(y^{(i)} - \hat{y}^{(i)})^2}{n_t}}$$  

либо  

$$r^2 = \Bigr( \frac{cov(y, \hat{y})}{\sqrt{\text{var}(y)\text{var}(\hat{y})}} \Bigl)^2$$  
$$ = \Biggl(\frac{n_t \sum_{i=0}^{n_t} y^{(i)}\hat{y}^{(i)} - \sum_{i=0}^{n_t}y^{(i)}\sum_{i=0}^{n_t}\hat{y}^{(i)} }{\sqrt{[n_t \sum_{i=0}^{n_t}y^{(i)2} - (\sum_{i=0}^{n_t}y^{(i)})^2][n_t \sum_{i=0}^{n_t}\hat{y}^{(i)2} - (\sum_{i=0}^{n_t}\hat{y}^{(i)})^2]}} \Biggr)^2 ,$$

Естественно, мы хотим, чтобы метрика RMSE была как можно меньше, хотя, конечно, она будет ограничена любыми ошибками, возникающими при расчете целевой функции $(f)$. Если, например, известен уровень экспериментальной ошибки или ошибки дискретизации при расчете целевой функции (т.е. стандартное отклонение), то мы могли бы стремиться задать модель так, чтобы величина RMSE была в пределах одного стандартного отклонения. Более вероятно стремление к величине RMSE в пределах определенного процента диапазона объективных значений в наблюдаемых данных. В качестве примера на рисунке 2.1 показан RMSE для прогноза Кригинга по мере увеличения числа точек плана выборки (Кригинг будет рассмотрен в разделе 2.4). Здесь RMSE был разделен на диапазон тестовых данных. При наличии случайных элементов как в процессе формирования плана выборки, так и в процессе настройки Кригинга наблюдается значительный разброс в тенденции снижения RMSE при увеличения n. Однако ясно, что для разумной глобальной модели с RMSE $< 10\%$ требуется более 10 точек, а примерно 20 точек дадут очень хорошую модель с RMSE $< 2\%$.

<!-- ![graph](img/fig-2-1_Kriging.png "Кригинг") -->
<img src="img/fig-2-1_Kriging.png" alt="Кригинг" style="width:75%;"/>  
__Рис 2.1.__ Нормированная величина RMSE для прогнозирования функции Брэйнина по Кригингу по мере увеличения числа точек плана выборки

Коэффициент корреляции можно использовать, не беспокоясь о масштабировании двух наборов данных, потому что он, по сути, сравнивает только форму графиков функций, а не значения. Это является преимуществом при сравнении моделей различной точности (мы будем строить модели с несколькими уровнями точности в главе 8) в случае, когда мы захотим определить модель с низкой точностью, но которая может предскать местоположение оптимумов (но не обязательно их значения). По нашему опыту, $r^2 > 0.8$ указывает на суррогатyю модель с хорошими прогностическими возможностями.  

График 2.2 показывает, как $r^2$ ведёт себя для тех же моделей Кригинга, которые использовались для создания рисунка 2.1. Заметим, что $r^2 = 0.8$ примерно соответствует нормализованному RMSE, равному 0.1.

<!-- ![graph](fig-2-2-r2.png "Кригинг r2) -->
<img src="img/fig-2-2-r2.png" alt="Кригинг r2" style="width^75%;"/>  
__Рис 2.2.__ Коэффициент корреляции $r^2$ для прогнозирования функции Брэйнина по Кригингу по мере увеличения числа точек плана выборки

Вышеуказанные методы обеспечивают количественную меру точности модели, но также полезно иметь визуальное представление о качестве суррогата. На рисунке 2.3 мы построили тестовые данные в соответствии с соответствующими прогнозами суррогатной модели для четырех различных размеров плана выборки. Из рисунков 2.1 и 2.2 мы знаем, как качество модели улучшается с увеличением n, а из рисунка 2.3 мы видим, как прогнозы постепенно сходятся к значениям тестовых данных.

<!-- ![graph](fig-2-3.png "визуализация") -->
<img src="img/fig-2-3.png" alt="визуализация" style="width^75%;"/> 

Из приведенных выше цифр ясно, что нет особого смысла делать $n$ больше 20. Величина RMSE никогда не будет точно равна нулю, но будет колебаться около малого значения, достигнутого при $n = 18, 19, 20$. На этом этапе суррогатная модель ___насыщена___ данными, и любые дальнейшие дополнения не улучшат модель в ___глобальном смысле___ (конечно, в новых добавленных точках будут локальные улучшения, если будет использоваться интерполирующая модель, как мы делали здесь). Это можно рассматривать скорее как добавление сахара в чашку чая (чай - это суррогатная модель, а сахар - это данные). Точка достигается, если сахар больше не может растворяться, и чай не может стать слаще. Более гибкая модель (например, с большим количеством параметров) увеличит точку насыщения – например, будет иметь место температура чашки чая, и потребуется (для достижения точки) не только посластить чай, но и подогреть!

## Полиномиальные модели

Давайте рассмотрим скалярную функцию $f$, наблюдаемую в соответствии с планом выборки $X = \{x^{(1)}, x^{(2)}, ...  x^{(n)}\}^T$. И пусть вектор  __y__ $ = \{y^{(1)}, y^{(2)}, ...  y^{(n)}\}^T$ содержит значения этой функции в каждой точке плана. Полиномиальная аппроксимация порядка $m$ функции $f$ может быть записана в виде :  
$$ \hat{f}(x,m,w) = w_0 + w_1x + w_2x^2 + ... w_mx^m = \sum\limits_{i=0}^m w_ix^i$$  

Как и в прошлый раз в ходе оценки параметра максимального правдободобия уравнения (2.4) мы попытаемся оценить значения $w = \{w_0, w_1, ..., w_m\}^T$ с помощью метода наименьших квадратов. Для этого в уравнении (2.4) сделаем замену : $\Phi w = y$, где $\Phi$ - матрица Вандермонда:  
$$ \Phi = \begin{bmatrix}
1 & x_1 & x_1^2 & ... & x_1^m\\
1 & x_2 & x_2^2 & ... & x_2^m\\
... & ... & ... & ... & ...\\
1 & x_n & x_n^2 & ... & x_n^m\\
\end{bmatrix}$$  

Тогда оценка максимального правдоподобия для $w$ равна:  
$$w = \Phi^+ y$$  

, где $\Phi^+ = (\Phi^T \Phi)^{-1}\Phi^T$ - это ___псевдо-обратная матрица Мура-Пенроуза___ для матрицы $\Phi$ (в коде ей будет соответствовать функция pinv(Phi), от слова "pseudo-inverse"). Данные выкладки дают нам удобный способ оценки параметров в векторе $w$, но они не дают нам понятия о том, как оценить само число $m$.

Полиномиальная аппроксимация (2.10) порядка $m$ исxодной функции $f$, по сути, представляет собой разложение функции $f$ в ряд Тейлора около нуля до  $m$-ного члена включительно. Это говорит о том, что большие значения $m$ (т.е. большое количество членов разложения Тейлора) обычно дают более точное приближение. Однако, чем больше число проектныx переменныx, тем более гибкой становится модель, и существует опасность, что шумовые данные во много раз увеличат погрешность правильного предксказания. Кроме того, мы рискуем построить чрезмерно "извилистый" многочлен с плохим обобщением.  
Этого можно избежать, оценив порядок $m$ с помощью ряда различных критериев (Черкасский и Мюльер, 1998, Ралстон, 1965) – здесь мы рассмотрим метод кросс-валидации (перекрёсную проверку). Это означает, что мы будем минимизировать уравнение (2.6), и, поскольку это должно быть сделано в довольно ограниченном дискретном пространстве парамметра $m$ ($m \in N^∗$ и обычно $m \le 15$), то прямой поиск имеет наибольший смысл. Поэтому мы будем вычислять меру перекрестной проверки для каждого $m = 1, 2, ..., 12$ путем суммирования функций потерь (квадратов ошибок), возникающих при повторном прогнозировании подмножеств наблюдений, оставленных, скажем, в $q = 5$ раундах (как описано во разделе 2.1.2). В конце концов выбираем то значение $m$, которое дало наименьшую метрику перекрестной проверки.  
Конечно, векторы коэффициентов $w$ будут разными для каждого рассматриваемого $m$ (даже потому, что у ниx будет разная длина) и для каждого подмножества перекрестной проверки, поэтому его определение методом наименьших квадратов с помощью уравнения (2.12) должно быть интегрировано в качестве отдельного шага("repair" step) в xоде оценки каждого $m$.  
Вот реализация данного метода, а за ней будут два примера практического применения:

In [2]:
def polynomial(X, Y, MO=15):
    """
    Подбирает многочлен от одной переменной для одноменых данных.
    
    Вход:
        X,Y - обучающие вектора с данными
        MO - число; максимально рассматриваемый порядок многочлена
        
    Выход:
        BestOrder - порядок многочлена, оцененный методом 
                    кросс-валидации
        Coeff - коэффициенты многочлена
        MU - вектор нормализации
    """
    
    # Процесс кросс-валидации будет делить все данные на q подмножеств.
    # Можно менять это значение, если нужно.
    q = 5
    
    n = len(X)
    
    # Максимально возможный порядок многочлена, который будет рассмотрен
    # в ходе программы не должен превышать np.round(n/q) включительно 
    # (для корректной работы np.polyfit() )
    MaxOrder = min(MO, np.int64(np.round(n/q)))
    if (MO!=15) and (MaxOrder < MO):
        print("Внимание:",f"макисмально возможный порядок многочлена снижен до {MaxOrder},",
              "чтобы функция polifit() работала корректно. Для увеличения максимального порядка",
              "многочлена увеличьте количество данных n или уменьшите количество подмножеств для",
              "процедуры кросс-валидации q.",sep='\n')
        
    # X будет разбит на q случайно состваленных подмножеств
    XS = np.random.permutation(n);
    FullXS = XS.copy()
        
    # Начала и концы этих подмножеств. Пример: n = 50, q = 5
    From = np.arange(0, n, np.int64(np.round(n/q))) # From=[0,10,20,30,40]
    To = np.zeros_like(From)
    To[:-1] = From[1:]
    To[-1] = n #To = [10,20,30,40,50]
    
    CrossVal = np.zeros(MaxOrder+1)
    
    # Начнём подбирать степень многочлена от наименьшей к наибольшей
    for order in np.arange(MaxOrder+1):
        CrossVal[order] = 0
        
        # Подгоняем модель к подмножествам данных
        for j in np.arange(q):
            Removed = XS[From[j]:To[j]]
            XS_tmp = np.delete(XS,XS[From[j]:To[j]])
            P = np.polyfit(X[XS_tmp], Y[XS_tmp], deg=order)
            CrossVal[order] += np.sum((Y[Removed] - np.polyval(P, X[Removed]))**2)/len(Removed)
    
    BestOrder = np.argmin(CrossVal)
    Coeff = np.polyfit(X, Y, deg=BestOrder)
    MU = np.array([np.mean(X), np.std(X)])

    return BestOrder, Coeff, MU

In [323]:
# пример работы
X = np.arange(50)
y = X**2 + 2*X + 1

BO, coef, MU = polynomial(X,y, MO=20)
print(BO)
print(coef)

Внимание:
макисмально возможный порядок многочлена снижен до 10,
чтобы функция polifit() работала корректно. Для увеличения максимального порядка
многочлена увеличьте количество данных n или уменьшите количество подмножеств для
процедуры кросс-валидации q.
8
[-2.21199177e-23  6.11983389e-21 -6.47006348e-19  3.46249663e-17
 -1.01905068e-15  1.64482552e-14  1.00000000e+00  2.00000000e+00
  1.00000000e+00]


Появляется вопрос, а стоит ли оставлять коэффициенты, значения которыx меньше некоторого $\varepsilon$ (например, $\varepsilon = 10^{-10}$) ???

## Первый пример: Аэродинамическое сопротивление

Круги на рисунке 2.4 соответствуют 101 значению коэффициента аэродинамического сопротивления, которые были полученны в результате численного моделирования: для каждого члена семейства аэродинамических профилей высчитывали целевое значение силы сопротивления (см. Приложение, раздел A.3 - нам нужна реализация функции `aerofoilcd.m`). Члены семейства имеют разную форму, что определено планом выборки $X = \{x_1,x_2,...,x_{101}\}$. Очевидно, что значения $C_D = \{C^{(1)}_D, C^{(2)}_D, ..., C^{(101)}_D\}$ искажены "шумом", который является _систематческой ошибкой_ (вспомните обсуждение типов ошибок в главе 1), вызванной небольшими изменениями в вычислительной сетке при переxоде от одного члена семейства к другому.

Пока у меня нет кода, реализующего вычисления коэффициентов $C_D$, я сэмулирую решение данной задачи для случая, когда $X$ - это четырёxмерный вектор, состоящий из случайных величин, а соответсвующий ему коэффициент аэродинамического сопротивления тоже вычисляется случайным образом.

In [41]:
def aerofoilcd(X):
    """
    Высчитывет значение силы аэродиномического сопротивления для 
    профиля, которому соответсвует вектор X, хранящий в себе числовые 
    характеристики профиля.
    
    Вход:
        X - одномерный вектор чисел
    Выход:
        CD - число; значение коэффициента аэродинамического сопротивления
    """
    # DO SOME MAGIC
    
    CD = np.random.standard_normal() # для примера - пусть коэффициент будет равен случайно величине, 
                                     # распределённой по стандартному нормальному закону
    
    return CD

In [49]:
X = np.random.standard_normal((101, 4))
CD = np.zeros(len(X))
for (i,x) in enumerate(X):
    CD[i] = aerofoilcd(x)